In [1]:
import pyhf
import numpy as np
import matplotlib.pyplot as plt
from pyhf.contrib.viz import brazil
import math

import ROOT
from ROOT import TGraph,TCanvas,TLegend,TGraphErrors

Welcome to JupyROOT 6.28/02


In [2]:
def plotUpperLimits(values,limits,medianobs_error,XaxisTitle,YaxisTitle,outfigure,theoryprediction = None,logy=False,logx=False):
    # see CMS plot guidelines: https://ghm.web.cern.ch/ghm/plots/
 
    N = len(values)
    yellow = TGraph(2*N)    # yellow band
    green = TGraph(2*N)     # green band
    median = TGraph(N)      # median line
    medianobs = TGraph(N)
    if theoryprediction:
        theory = TGraph(N)      # theory line
 
    up2s = [ ]
    dn2s = [ ]
    for i in range(N):
        limit = limits[i]
        up2s.append(limit[4])
        dn2s.append(limit[0])
        yellow.SetPoint(    i,    values[i], limit[4] ) # + 2 sigma
        green.SetPoint(     i,    values[i], limit[3] ) # + 1 sigma
        median.SetPoint(    i,    values[i], limit[2] ) # median
        medianobs.SetPoint(    i,    values[i], limit[5] ) # medianobs
        if theoryprediction:
            theory.SetPoint(    i,    values[i], theoryprediction[i] ) # median
        green.SetPoint(  2*N-1-i, values[i], limit[1] ) # - 1 sigma
        yellow.SetPoint( 2*N-1-i, values[i], limit[0] ) # - 2 sigma
 
    # Create a TGraphErrors for medianobs with errors
    if medianobs_error is not None:
        medianobs_errors = TGraphErrors(N)
        for i in range(N):
            medianobs_errors.SetPoint(i, values[i], medianobs.GetY()[i])
            medianobs_errors.SetPointError(i,0, medianobs_error[i])  # Set vertical error for each point

    W = 800
    H  = 800
    T = 0.08*H
    B = 0.12*H
    L = 0.16*W
    R = 0.04*W
    c = TCanvas("c","c",100,100,W,H)
    c.SetFillColor(0)
    c.SetBorderMode(0)
    c.SetFrameFillStyle(0)
    c.SetFrameBorderMode(0)
    c.SetLeftMargin( L/W )
    c.SetRightMargin( R/W )
    c.SetTopMargin( T/H )
    c.SetBottomMargin( B/H )
    c.SetTickx(0)
    c.SetTicky(0)
    c.SetGrid()
    if logy:
        c.SetLogy()
    if logx:
        c.SetLogx()
    c.cd()
    
    frame = c.DrawFrame(min(values)*0.8,min(dn2s)*0.001, max(values)*2, max(up2s)*10)
    frame.GetYaxis().CenterTitle()
    frame.GetYaxis().SetTitleSize(0.05)
    frame.GetXaxis().SetTitleSize(0.05)
    frame.GetXaxis().SetLabelSize(0.03)
    frame.GetYaxis().SetLabelSize(0.03)
    frame.GetYaxis().SetTitleOffset(1.5)
    frame.GetXaxis().SetNdivisions(508)
    frame.GetYaxis().CenterTitle(True)
    frame.GetYaxis().SetTitle(YaxisTitle)
#    frame.GetYaxis().SetTitle("95% upper limit on #sigma #times BR / (#sigma #times BR)_{SM}")
    frame.GetXaxis().SetTitle(XaxisTitle)
    if logy:
        frame.SetMinimum(min(dn2s)*0.1)
        frame.SetMaximum(max(up2s)*10)
    else:
        frame.SetMinimum(0)
        frame.SetMaximum(max(up2s)*1.2)
    frame.GetXaxis().SetLimits(min(values),max(values))
 
    yellow.SetFillColor(ROOT.kOrange)
    yellow.SetLineColor(ROOT.kOrange)
    yellow.SetFillStyle(1001)
    yellow.Draw('F')
 
    green.SetFillColor(ROOT.kGreen+1)
    green.SetLineColor(ROOT.kGreen+1)
    green.SetFillStyle(1001)
    green.Draw('Fsame')

    medianobs.SetLineColor(1)
    medianobs.SetLineWidth(4)
    medianobs.SetLineStyle(1)
    medianobs.Draw('Lsame')
 
    median.SetLineColor(2)
    median.SetLineWidth(3)
    median.SetLineStyle(2)
    median.Draw('Lsame')

    if theoryprediction:
        theory.SetLineColor(2)
        theory.SetLineWidth(6)
        theory.SetLineStyle(1)
        theory.Draw('Lsame')
 
    # Draw the medianobs error bars (cross-shaped, with size proportional to errors)
    if medianobs_error is not None:
        medianobs_errors = TGraphErrors(N)
        for i in range(N):
            medianobs_errors.SetPoint(i, values[i], medianobs.GetY()[i])
            # 设置误差：X方向误差=0（不需要），Y方向误差=medianobs_error[i]
            medianobs_errors.SetPointError(i, 0, medianobs_error[i])

        # 设置十字形误差棒样式
        medianobs_errors.SetMarkerStyle(20)          # 中心点样式（实心圆）
        medianobs_errors.SetMarkerColor(ROOT.kBlack)   # 中心点颜色
        medianobs_errors.SetLineColor(ROOT.kBlack)     # 误差棒颜色
        medianobs_errors.SetLineWidth(3)             # 误差棒线宽
        medianobs_errors.SetMarkerSize(0)            # 隐藏中心点（仅显示十字）
        medianobs_errors.Draw("PE same")             # "PE"表示点+误差棒
    print("Debug: errors:", [medianobs_error[i] for i in range(N)])

#     CMS_lumi.CMS_lumi(c,14,11)
    ROOT.gPad.SetTicks(1,1)
    frame.Draw('sameaxis')
 
    x1 = 0.1
    x2 = 0.3
    y2 = 0.9
    y1 = 0.7
    legend = TLegend(0.2,0.7,0.5,0.9)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.03)
    legend.SetTextFont(42)
    legend.AddEntry(median, "Asymptotic CL_{s} expected",'L')
    legend.AddEntry(medianobs, "Asymptotic CL_{s} observed",'lep')
    legend.AddEntry(green, "#pm 1 std. deviation",'f')
#    legend.AddEntry(green, "Asymptotic CL_{s} #pm 1 std. deviation",'f')
    legend.AddEntry(yellow,"#pm 2 std. deviation",'f')
    if theoryprediction:
        legend.AddEntry(theory, "theory prediciotn",'L')
#    legend.AddEntry(green, "Asymptotic CL_{s} #pm 2 std. deviation",'f')
    legend.Draw()
    
    c.Update()
    c.Print(outfigure)
 
    Graphs = [yellow,median,green,]
    if theoryprediction:
        Graphs = [yellow,median,green,theory,legend]

    return c,Graphs


In [3]:
limits_const_air_obs=[]
limits_error=[]
#MDM=[0.001,0.01,0.1,1,10,100,1000,10000,100000]
#DMmass=['0p001','0p01','0p1','1','10','100','1000','10000','100000']
#MDM=[0.01,0.1,1,10,100]
#DMmass=['0p01','0p1','1','10','100']
MDM=[0.01,0.05,0.1,0.5,1,5,10,50,100]
DMmass=['0p01','0p05','0p1','0p5','1','5','10','50','100']

#const_air
for iDMmass in DMmass:
    flimits = ROOT.TFile.Open("UL_const_air_obs/higgsCombine" + "const_" + iDMmass + "_air_obs" + ".AsymptoticLimits.mH120.root")
    tlimits = flimits.Get("limit")
    limitsnow=[]
    
    nEvents = tlimits.GetEntries()
    for i in range(nEvents):
        tlimits.GetEntry(i)
        limitsnow.append(tlimits.limit)
        if i == 5:
            limits_error.append(tlimits.limitErr)
    
    limits_const_air_obs.append(limitsnow)
    flimits.Close()

    

nsigHist_const=[]
fHist = ROOT.TFile.Open("./Hist.root")
for iDMmass in DMmass:
    Hist = fHist.Get("h_const_"+iDMmass)
    nsigHist_const.append(Hist.Integral())
#nbkgHist_air=fHist.Get("h_air").Integral()
nbkgHist_obs=fHist.Get("h_obs").Integral()

In [4]:
#Dark Matter
rhoDM=0.3#GeV/cm^3
Vbox=22*22*22#cm^3
A = 22*22#cm^2

NDM=[]
for m in MDM:
    NDM.append(m*A/rhoDM/Vbox)
    
#signal
#Nsig/eff=NDM*I*sigma*T
epsilon=[0.6672, 0.6709, 0.6712, 0.6916, 0.6723, 0.6382, 0.6382, 0.6380, 0.6267]
epsilon_sigma=[0.0130, 0.0134, 0.0131, 0.0131, 0.0132, 0.0131, 0.0123, 0.0125, 0.0125]

#epsilon=[0.4302, 0.4307, 0.4330, 0.4456, 0.4260, 0.4007, 0.4325, 0.4391, 0.4002]
#epsilon_sigma=[0.0057, 0.0058, 0.0058, 0.0059, 0.0057, 0.0056, 0.0057, 0.0059, 0.0055]

#epsilon=[0.2980, 0.2974, 0.2979, 0.3025, 0.2959, 0.2800, 0.2964, 0.2990, 0.2846]
#epsilon_sigma=[0.0028, 0.0028, 0.0028, 0.0029, 0.0028, 0.0028, 0.0028, 0.0028, 0.0028]

#epsilon=[0.3730, 0.3744, 0.3778, 0.3828, 0.3722, 0.3605, 0.3724, 0.3774, 0.3600]
#epsilon_sigma=[0.0037, 0.0038, 0.0039, 0.0039, 0.0038, 0.0038, 0.0038, 0.0038, 0.0037]

#epsilon=[0.3907, 0.3833, 0.3940, 0.3960, 0.3851, 0.3714, 0.3904, 0.3926, 0.3760]
#epsilon_sigma=[0.0041, 0.0041, 0.0043, 0.0042, 0.0042, 0.0041, 0.0042, 0.0042, 0.0041]

eff = 0.7132
eff_sigma = 0.0257
#for i in range(len(nsigHist_const)):
#    eff=0.75*0.75*0.75*0.75
    
#    eff_const.append(eff)

    

I=1/60. # s^-1 cm^-2
T=5380240 #s ≈ 63d
#eff_obs=147251./2286807.
fE = 10e-15

sigmalimits_const_air=[]
sigmalimits_const_air_error=[]
sigmalimits_const_vac=[]
sigmalimits_maxwell_air=[]
sigmalimits_maxwell_vac=[]

#const_air
for iDM in range(len(DMmass)):
    limitsnow=[]
    limits_errornow=[]
    for iCL in range(6):
        if iCL <= 4:
            limitsnow.append(NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / epsilon[iDM] * limits_const_air_obs[iDM][iCL] * fE)
        if iCL == 5:
            limitsnow.append(NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / epsilon[iDM] / eff * limits_const_air_obs[iDM][iCL] * fE)
            limits_errornow.append((((NDM[iDM] / nbkgHist_obs / epsilon[iDM] /eff * limits_const_air_obs[iDM][iCL] * fE * ((nsigHist_const[iDM])**(0.5)))**(2)) + (NDM[iDM] * nsigHist_const[iDM] / epsilon[iDM] / eff * limits_const_air_obs[iDM][iCL] * fE * ((nbkgHist_obs)**(-2)) * ((nbkgHist_obs)**(0.5)))**(2) + (NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / eff * limits_const_air_obs[iDM][iCL] * fE * ((epsilon[iDM])**(-2)) * epsilon_sigma[iDM])**(2)+ (NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / epsilon[iDM] * limits_const_air_obs[iDM][iCL] * fE * ((eff)**(-2)) * eff_sigma)**(2) + (NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / epsilon[iDM] / eff * fE * limits_error[iDM])**(2))**(0.5))
            #limits_errornow.append(((NDM[iDM] / nbkgHist_obs / epsilon[iDM] / eff * limits_const_air_obs[iDM][iCL] * fE * ((nsigHist_const[iDM])**(0.5)))**(2)) + (NDM[iDM] * nsigHist_const[iDM] / epsilon[iDM] / eff * limits_const_air_obs[iDM][iCL] * fE * ((nbkgHist_obs)**(-2)) * ((nbkgHist_obs)**(0.5)))**(2) + (NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / eff * limits_const_air_obs[iDM][iCL] * fE * ((epsilon[iDM])**(-2)) * epsilon_sigma[iDM])**(2) + (NDM[iDM] * nsigHist_const[iDM] / nbkgHist_obs / epsilon[iDM] * limits_const_air_obs[iDM][iCL] * fE * ((eff)**(-2)) * eff_sigma)**(2))
    sigmalimits_const_air.append(limitsnow)
    sigmalimits_const_air_error.append(limits_errornow[0])

print(sigmalimits_const_air_error)
print(sigmalimits_const_air[4])
    

[2.4348861353558847e-20, 1.1411158000363266e-19, 1.7975087757538554e-19, 6.48805041421195e-19, 1.032563231114611e-18, 8.624324483496841e-18, 1.463041177352381e-17, 1.2780845786438844e-16, 1.6329551748597517e-16]
[1.2975959986515272e-17, 1.7310095172228596e-17, 2.4071346061941378e-17, 3.347438815669735e-17, 4.4646496712025926e-17, 1.8155521230796554e-17]


In [5]:
can,_ = plotUpperLimits(MDM,sigmalimits_const_air,sigmalimits_const_air_error,"#it{M}_{DM} (GeV)","95% upper limit on #it{#sigma}_{#mu,DM} / #it{f}_{E} (cm^{2})","figure/UL_const_air_obs.eps",logy=True,logx=True)

Debug: errors: [2.4348861353558847e-20, 1.1411158000363266e-19, 1.7975087757538554e-19, 6.48805041421195e-19, 1.032563231114611e-18, 8.624324483496841e-18, 1.463041177352381e-17, 1.2780845786438844e-16, 1.6329551748597517e-16]


Info in <TCanvas::Print>: eps file figure/UL_const_air_obs.eps has been created


In [6]:
NDM

[0.0015151515151515152,
 0.007575757575757578,
 0.015151515151515155,
 0.07575757575757576,
 0.15151515151515152,
 0.7575757575757576,
 1.5151515151515151,
 7.575757575757576,
 15.151515151515152]

In [7]:
nsigHist_const

[1898.2605411410332,
 2001.009264945984,
 1527.2448601722717,
 1037.1694993972778,
 660.4569473266602,
 418.787249982357,
 274.17863619327545,
 275.1957746744156,
 183.49679589271545]

In [8]:
nbkgHist_obs

33937.0

In [9]:
epsilon

[0.6672, 0.6709, 0.6712, 0.6916, 0.6723, 0.6382, 0.6382, 0.638, 0.6267]

In [10]:
limits_const_air_obs[0][5]

0.2882769218463271

In [11]:
epsilon_sigma

[0.013, 0.0134, 0.0131, 0.0131, 0.0132, 0.0131, 0.0123, 0.0125, 0.0125]

In [12]:
limits_error

[0.0011022335745550782,
 0.0008363572296409827,
 0.0008458316235576413,
 0.0008156539329538248,
 0.0007806995808516959,
 0.002868662228718788,
 0.003284268322807704,
 0.0046253506922989684,
 0.0032908906906978785]

In [13]:
#eff_maxwell